In [1]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.1
SparkUI available at http://wm598-921.fios-router.home:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /Users/shifa/dev/hail_elasticsearch_pipelines/genome_sv_pipeline/hail-20210405-2000-0.2.61-3c86d3ba497a.log


In [2]:
from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()

Loading BokehJS ...

In [3]:
# hl.import_vcf('vcf/sv.vcf.gz', force=True, reference_genome='GRCh38').write('vcf/svs.mt', overwrite=True)

In [3]:
mt = hl.read_matrix_table('vcf/svs.mt')

In [4]:
mt.count()

(145568, 714)

In [5]:
import subprocess
GS_SAMPLE_PATH = 'gs://seqr-datasets/v02/GRCh38/RDG_{sample_type}_Broad_Internal/base/projects/{project_guid}/{project_guid}_{file_ext}'
def _get_gs_samples(project_guid, file_ext, expected_header, sample_type):
    """
    Get sample metadata from files in google cloud

    :param project_guid: seqr project identifier
    :param file_ext: extension for the desired sample file
    :param expected_header: expected header to validate file
    :param sample_type: sample type (WES/WGS)
    :return: parsed data from the sample file as a list of lists
    """
    file = GS_SAMPLE_PATH.format(project_guid=project_guid, sample_type=sample_type, file_ext=file_ext)
    process = subprocess.Popen(
        'gsutil cat {}'.format(file), stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True)
    if process.wait() != 0:
        return None
    header = next(process.stdout).decode('utf-8')
    if header.strip() != expected_header:
        raise Exception('Missing header for sample file, expected "{}" but found {}'.format(
            expected_header, header))
    return [line.decode('utf-8').strip().split('\t') for line in process.stdout]


def get_sample_subset(project_guid, sample_type):
    """
    Get sample id subset for a given project

    :param project_guid: seqr project identifier
    :param sample_type: sample type (WES/WGS)
    :return: set of sample ids
    """
    subset = _get_gs_samples(project_guid, file_ext='ids.txt', sample_type=sample_type, expected_header='s')
    if not subset:
        raise Exception('No sample subset file found')
    return {row[0] for row in subset}

<br/>
Subset the samples to the project `R0332_cmg_estonia_wgs`

In [6]:
sample_subset = get_sample_subset('R0332_cmg_estonia_wgs', 'WGS')
subset = hl.literal(sample_subset)
mt1 = mt.filter_cols(subset.contains(mt['s']))
missing_samples = sample_subset - {col.s for col in mt1.key_cols_by().cols().collect()}
print('{} missing samples and the first 10 of them are: {}'.format(len(missing_samples), list(missing_samples)[:10]))

61 missing samples and the first 10 of them are: ['HK061-0159_D1', 'OUN_HK124_001_D1', 'HK060-0156_1', 'HK115-002_1', 'HK079-001_D2', 'OUN_HK126_001_D1', 'OUN_HK124_003_D1', 'HK061-0158_D1', 'HK060-0155_1', 'HK112-003_1']


In [7]:
mt1.GT[0].show()

,,,,,,
,,'HK010_0026','HK015_0037','HK031_0079','HK031_0080','HK069-0178_1'
locus,alleles,,,,,
locus<GRCh38>,array<str>,int32,int32,int32,int32,int32
chr1:10000,"[""N"",""<DUP>""]",0,0,0,1,0
chr1:10000,"[""N"",""<DUP>""]",0,1,0,0,0
chr1:10602,"[""N"",""<BND>""]",0,0,0,0,0
chr1:41950,"[""N"",""<DUP>""]",0,0,0,0,0
chr1:44000,"[""N"",""<DUP>""]",0,0,0,0,0
chr1:44250,"[""N"",""<DUP>""]",0,0,0,0,0
chr1:51400,"[""N"",""<DEL>""]",0,0,0,0,0


In [8]:
samples = hl.agg.filter(mt1.GT.is_non_ref(), hl.agg.collect(hl.struct(id=mt1.s, gq=mt1.GQ, num_alt=mt1.GT[0]+mt1.GT[1], cn=mt1.RD_CN)))
mt2a = mt1.annotate_rows(samples=samples)
mt2 = mt2a.filter_rows(mt2a.samples != hl.empty_array(hl.dtype('struct{id: str, gq: int32, num_alt: int32, cn: int32}')))
mt2.count()

(67275, 106)

In [9]:
mtx = mt2.filter_entries(hl.is_defined(mt2.RD_CN))
rd_cn_cnt = sorted(mtx.aggregate_entries(hl.agg.counter(mtx.RD_CN)).items())
print('{} different RD_CNs, the counter for the first 10 RD_CNs are: {}.'.format(len(rd_cn_cnt), rd_cn_cnt[:10]))
print('Total RD_CNs>9: {}, max RD_CN: {}'.format(sum(rd_cn_cnt[10:][1]), rd_cn_cnt[-1][0]))

1181 different RD_CNs, the counter for the first 10 RD_CNs are: [(0, 51131), (1, 425325), (2, 4177497), (3, 315015), (4, 79143), (5, 32265), (6, 18043), (7, 11620), (8, 7940), (9, 6123)].
Total RD_CNs>9: 3797, max RD_CN: 4066


<br/>
The distributions of the values of the RD_CNs

In [10]:
dp_hist = mt2.aggregate_entries(hl.expr.aggregators.hist(mt2.RD_CN, start=0, end=10, bins=11))
p = hl.plot.histogram(dp_hist, legend='RD_CN', title='RD_CN Histogram')
show(p)

<br/>
Statistics for differet GT values

In [11]:
mt2.aggregate_entries(hl.agg.counter(mt2.GT))

{Call(alleles=[0, 0], phased=False): 6033770,
 Call(alleles=[0, 1], phased=False): 932341,
 Call(alleles=[1, 1], phased=False): 157472,
 None: 7567}

<br/>
RD_CN distributions for different GTs:

In [16]:
cnts={}
mtx = mt2.filter_entries(hl.is_defined(mt2.RD_CN))
for alts in range(3):
    field_name = 'GT'+str(alts)+'_RD_CN'
    mtx = mtx.annotate_entries(**{field_name: hl.if_else(mtx.GT.n_alt_alleles()==alts, mtx.RD_CN, -1)})
    cnts = sorted(mtx.aggregate_entries(hl.agg.counter(mtx[field_name])).items())
    cnts[11] = ('>9', sum(cnts[11:][1]))
    print('{}: {}'.format(field_name, cnts[:12]))

for field_name in ['GT0_RD_CN', 'GT1_RD_CN', 'GT2_RD_CN']:
    mtxx = mtx.filter_entries(mtx[field_name]>=0)
    dp_hist = mtxx.aggregate_entries(hl.expr.aggregators.hist(mtxx[field_name], start=0, end=10, bins=11))
    p = hl.plot.histogram(dp_hist, legend=field_name, title=field_name+' Histogram')
    show(p)

GT0_RD_CN: [(-1, 660908), (0, 8490), (1, 223817), (2, 3953419), (3, 199083), (4, 43557), (5, 20124), (6, 12082), (7, 7929), (8, 5431), (9, 4435), ('>9', 2885)]
GT1_RD_CN: [(-1, 4624911), (0, 14630), (1, 166020), (2, 206596), (3, 108528), (4, 22531), (5, 7503), (6, 3832), (7, 2535), (8, 1809), (9, 1244), ('>9', 668)]
GT2_RD_CN: [(-1, 5050345), (0, 28011), (1, 35488), (2, 17482), (3, 7404), (4, 13055), (5, 4638), (6, 2129), (7, 1156), (8, 700), (9, 444), ('>9', 266)]


In [22]:
mtx = mt2.filter_entries(hl.is_defined(mt2.RD_CN))
max_cn = 5
for cn in range(max_cn):
    field_name = 'Counters for RD_CN={} per genotype and SV type:'.format(cn) if cn<max_cn-1 else 'Counters for RD_CN>={} per genotype and SV type'.format(cn)
    mtx = mtx.annotate_entries(**{field_name: hl.if_else(mtx.RD_CN==cn if cn<max_cn-1 else mtx.RD_CN>=cn , hl.struct(alts=mtx.GT.n_alt_alleles(), svtype=mtx.info.SVTYPE),
                                                         hl.struct(alts=-1, svtype='NA'))})
    cnts = mtx.aggregate_entries(hl.agg.counter(mtx[field_name]))
    cnts = sorted(cnts.items(), key=lambda item: item[0].alts)
    pprint((field_name, cnts[1:]))

('Counters for RD_CN=0 per genotype and SV type:',
 [(Struct(alts=0, svtype='DUP'), 3171),
  (Struct(alts=0, svtype='CPX'), 4),
  (Struct(alts=0, svtype='INS'), 16),
  (Struct(alts=0, svtype='DEL'), 5299),
  (Struct(alts=1, svtype='DUP'), 116),
  (Struct(alts=1, svtype='DEL'), 14487),
  (Struct(alts=1, svtype='INS'), 27),
  (Struct(alts=2, svtype='DUP'), 8),
  (Struct(alts=2, svtype='DEL'), 28002),
  (Struct(alts=2, svtype='CPX'), 1)])
('Counters for RD_CN=1 per genotype and SV type:',
 [(Struct(alts=0, svtype='DUP'), 86321),
  (Struct(alts=0, svtype='CPX'), 191),
  (Struct(alts=0, svtype='INS'), 402),
  (Struct(alts=0, svtype='DEL'), 136903),
  (Struct(alts=1, svtype='DUP'), 2011),
  (Struct(alts=1, svtype='DEL'), 163754),
  (Struct(alts=1, svtype='CPX'), 43),
  (Struct(alts=1, svtype='INS'), 212),
  (Struct(alts=2, svtype='DUP'), 11),
  (Struct(alts=2, svtype='DEL'), 35471),
  (Struct(alts=2, svtype='INS'), 6)])
('Counters for RD_CN=2 per genotype and SV type:',
 [(Struct(alts=0, svt

In [13]:
rows = mt1.rows()
rows.show()

+---------------+---------------+-----------------------------+----------+
| locus         | alleles       | rsid                        |     qual |
+---------------+---------------+-----------------------------+----------+
| locus<GRCh38> | array<str>    | str                         |  float64 |
+---------------+---------------+-----------------------------+----------+
| chr1:10000    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_1" | 9.99e+02 |
| chr1:10000    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_2" | 9.99e+02 |
| chr1:10602    | ["N","<BND>"] | "CMG.phase1_CMG_BND_chr1_1" | 4.61e+02 |
| chr1:41950    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_3" | 9.99e+02 |
| chr1:44000    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_4" | 9.99e+02 |
| chr1:44250    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_5" | 9.99e+02 |
| chr1:51400    | ["N","<DEL>"] | "CMG.phase1_CMG_DEL_chr1_1" | 9.99e+02 |
| chr1:52600    | ["N","<CNV>"] | "CMG.phase1_CMG_CNV_chr1_1" | 9.99e+02 |
| chr1:66234    | ["N","<BND>"] | "CMG.phase1_CMG_BND_chr1_2" | 8.07e+02 |
| chr1:66350    | ["N","<DEL>"] | "CMG.phase1_CMG_DEL_chr1_2" | 7.74e+02 |
+---------------+---------------+-----------------------------+----------+

+------------------------------------------+-----------------+-----------+--------------------+
| filters                                  | info.ALGORITHMS | info.CHR2 | info.CPX_INTERVALS |
+------------------------------------------+-----------------+-----------+--------------------+
| set<str>                                 | array<str>      | str       | array<str>         |
+------------------------------------------+-----------------+-----------+--------------------+
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"UNRESOLVED","UNSTABLE_AF_PCRMINUS"}    | ["manta"]       | "chr12"   | NA                 |
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"UNSTABLE_AF_PCRMINUS","LOW_CALL_RATE"} | ["depth"]       | "chr1"    | NA                 |
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"FAIL_minGQ"}                           | ["depth"]       | "chr1"    | NA                 |
| {"UNRESOLVED"}                           | ["manta"]       | "chr19"   | NA                 |
| {"FAIL_OUTLIER_REMOVAL"}                 | ["manta"]       | "chr1"    | NA                 |
+------------------------------------------+-----------------+-----------+--------------------+

+---------------+----------+-----------+---------------+-------------+--------------+------------+
| info.CPX_TYPE | info.END | info.END2 | info.EVIDENCE | info.SOURCE | info.STRANDS | info.SVLEN |
+---------------+----------+-----------+---------------+-------------+--------------+------------+
| str           |    int32 |     int32 | array<str>    | str         | str          |      int32 |
+---------------+----------+-----------+---------------+-------------+--------------+------------+
| NA            |    17000 |        NA | ["RD"]        | NA          | NA           |       7000 |
| NA            |    53500 |        NA | ["BAF","RD"]  | NA          | NA           |      43500 |
| NA            |    10602 |     10546 | ["SR"]        | NA          | "+-"         |         -1 |
| NA            |    52000 |        NA | ["BAF","RD"]  | NA          | NA           |      10050 |
| NA            |    66000 |        NA | ["RD"]        | NA          | NA           |      22000 |
| NA            |   116000 |        NA | ["BAF","RD"]  | NA          | NA           |      71750 |
| NA            |    64000 |        NA | ["RD"]        | NA          | NA           |      12600 |
| NA            |    58000 |        NA | ["RD"]        | NA          | NA 

In [14]:
rows.info.get('PROTEIN_CODING__COPY_GAIN').dtype == hl.dtype('array<str>')

True

In [15]:
r1 = rows.annotate(t=hl.if_else(hl.is_defined(rows.info.PROTEIN_CODING__COPY_GAIN),
                           hl.len(rows.info.PROTEIN_CODING__COPY_GAIN), 0))
r1.show()

+---------------+---------------+-----------------------------+----------+
| locus         | alleles       | rsid                        |     qual |
+---------------+---------------+-----------------------------+----------+
| locus<GRCh38> | array<str>    | str                         |  float64 |
+---------------+---------------+-----------------------------+----------+
| chr1:10000    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_1" | 9.99e+02 |
| chr1:10000    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_2" | 9.99e+02 |
| chr1:10602    | ["N","<BND>"] | "CMG.phase1_CMG_BND_chr1_1" | 4.61e+02 |
| chr1:41950    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_3" | 9.99e+02 |
| chr1:44000    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_4" | 9.99e+02 |
| chr1:44250    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_5" | 9.99e+02 |
| chr1:51400    | ["N","<DEL>"] | "CMG.phase1_CMG_DEL_chr1_1" | 9.99e+02 |
| chr1:52600    | ["N","<CNV>"] | "CMG.phase1_CMG_CNV_chr1_1" | 9.99e+02 |
| chr1:66234    | ["N","<BND>"] | "CMG.phase1_CMG_BND_chr1_2" | 8.07e+02 |
| chr1:66350    | ["N","<DEL>"] | "CMG.phase1_CMG_DEL_chr1_2" | 7.74e+02 |
+---------------+---------------+-----------------------------+----------+

+------------------------------------------+-----------------+-----------+--------------------+
| filters                                  | info.ALGORITHMS | info.CHR2 | info.CPX_INTERVALS |
+------------------------------------------+-----------------+-----------+--------------------+
| set<str>                                 | array<str>      | str       | array<str>         |
+------------------------------------------+-----------------+-----------+--------------------+
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"UNRESOLVED","UNSTABLE_AF_PCRMINUS"}    | ["manta"]       | "chr12"   | NA                 |
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"UNSTABLE_AF_PCRMINUS","LOW_CALL_RATE"} | ["depth"]       | "chr1"    | NA                 |
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"FAIL_minGQ"}                           | ["depth"]       | "chr1"    | NA                 |
| {"UNRESOLVED"}                           | ["manta"]       | "chr19"   | NA                 |
| {"FAIL_OUTLIER_REMOVAL"}                 | ["manta"]       | "chr1"    | NA                 |
+------------------------------------------+-----------------+-----------+--------------------+

+---------------+----------+-----------+---------------+-------------+--------------+------------+
| info.CPX_TYPE | info.END | info.END2 | info.EVIDENCE | info.SOURCE | info.STRANDS | info.SVLEN |
+---------------+----------+-----------+---------------+-------------+--------------+------------+
| str           |    int32 |     int32 | array<str>    | str         | str          |      int32 |
+---------------+----------+-----------+---------------+-------------+--------------+------------+
| NA            |    17000 |        NA | ["RD"]        | NA          | NA           |       7000 |
| NA            |    53500 |        NA | ["BAF","RD"]  | NA          | NA           |      43500 |
| NA            |    10602 |     10546 | ["SR"]        | NA          | "+-"         |         -1 |
| NA            |    52000 |        NA | ["BAF","RD"]  | NA          | NA           |      10050 |
| NA            |    66000 |        NA | ["RD"]        | NA          | NA           |      22000 |
| NA            |   116000 |        NA | ["BAF","RD"]  | NA          | NA           |      71750 |
| NA            |    64000 |        NA | ["RD"]        | NA          | NA           |      12600 |
| NA            |    58000 |        NA | ["RD"]        | NA          | NA 

In [16]:
rows.info.PROTEIN_CODING__COPY_GAIN.dtype

dtype('array<str>')

In [17]:
gene_cols = [gene_col for gene_col in rows.info if gene_col.startswith('PROTEIN_CODING__') and rows.info.get(gene_col).dtype==hl.dtype('array<str>')]
print(gene_cols)
r1 = rows.annotate(t=[hl.if_else(hl.is_defined(rows.info.get(gene)),
                           hl.len(rows.info.get(gene)), 0) for gene in gene_cols])
r1.aggregate(hl.agg.array_agg(lambda e: hl.agg.max(e), r1.t))

['PROTEIN_CODING__LOF', 'PROTEIN_CODING__DUP_LOF', 'PROTEIN_CODING__COPY_GAIN', 'PROTEIN_CODING__DUP_PARTIAL', 'PROTEIN_CODING__MSV_EXON_OVR', 'PROTEIN_CODING__INTRONIC', 'PROTEIN_CODING__INV_SPAN', 'PROTEIN_CODING__UTR', 'PROTEIN_CODING__NEAREST_TSS', 'PROTEIN_CODING__PROMOTER']


[1622, 3, 161, 22, 8, 84, 174, 2, 2, 8]

In [18]:
r2 = r1.filter(r1.t[0]>2)
r2.show()

+----------------+---------------+--------------------------------+----------+
| locus          | alleles       | rsid                           |     qual |
+----------------+---------------+--------------------------------+----------+
| locus<GRCh38>  | array<str>    | str                            |  float64 |
+----------------+---------------+--------------------------------+----------+
| chr1:12799082  | ["N","<DEL>"] | "CMG.phase1_CMG_DEL_chr1_333"  | 3.05e+02 |
| chr1:12838726  | ["N","<DEL>"] | "CMG.phase1_CMG_DEL_chr1_338"  | 9.99e+02 |
| chr1:13008000  | ["N","<DEL>"] | "CMG.phase1_CMG_DEL_chr1_344"  | 1.00e+00 |
| chr1:13074384  | ["N","<DEL>"] | "CMG.phase1_CMG_DEL_chr1_347"  | 9.99e+02 |
| chr1:13260192  | ["N","<DEL>"] | "CMG.phase1_CMG_DEL_chr1_352"  | 1.43e+02 |
| chr1:16619297  | ["N","<CPX>"] | "CMG.phase1_CMG_CPX_chr1_12"   | 9.99e+02 |
| chr1:50768786  | ["N","<DEL>"] | "CMG.phase1_CMG_DEL_chr1_977"  | 1.00e+00 |
| chr1:145510587 | ["N","<DEL>"] | "CMG.phase1_CMG_DEL_chr1_3101" | 9.99e+02 |
| chr1:145822587 | ["N","<DEL>"] | "CMG.phase1_CMG_DEL_chr1_3107" | 9.99e+02 |
| chr1:204145612 | ["N","<DEL>"] | "CMG.phase1_CMG_DEL_chr1_4603" | 9.99e+02 |
+----------------+---------------+--------------------------------+----------+

+--------------------------+-------------------+-----------+
| filters                  | info.ALGORITHMS   | info.CHR2 |
+--------------------------+-------------------+-----------+
| set<str>                 | array<str>        | str       |
+--------------------------+-------------------+-----------+
| {}                       | ["depth","wham"]  | "chr1"    |
| {}                       | ["depth","manta"] | "chr1"    |
| {}                       | ["depth"]         | "chr1"    |
| {"FAIL_OUTLIER_REMOVAL"} | ["depth"]         | "chr1"    |
| {}                       | ["depth"]         | "chr1"    |
| {}                       | ["manta"]         | "chr1"    |
| {}                       | ["depth","manta"] | "chr1"    |
| {}                       | ["depth"]         | "chr1"    |
| {}                       | ["depth"]         | "chr1"    |
| {}                       | ["depth","manta"] | "chr1"    |
+--------------------------+-------------------+-----------+

+-------------------------------------------------------------------------------------------+
| info.CPX_INTERVALS                                                                        |
+-------------------------------------------------------------------------------------------+
| array<str>                                                                                |
+-------------------------------------------------------------------------------------------+
| NA                                                                                        |
| NA                                                                                        |
| NA                                                                                        |
| NA                                                                                        |
| NA                                                                                        |
| ["DEL_chr1:16619297-16725244","INV_chr1:16725244-16951955","DEL_chr1:16951955-234776440"] |
| NA                                                                                        |
| NA                                                                                        |
| NA                                                                                        |
| NA                                                                                        |
+-------------------------------------------------------------------------------------------+

+---------------+-----------+-----------+------------------+-------------+--------------+
| info.CPX_TYPE |  info.END | info.END2 | info.EVIDENCE    | info.SOURCE | info.STRANDS |
+---------------+-----------+-----------+------------------+-------------

In [19]:
rows.aggreate(hl.agg.count_where(rows.info.get()))

AttributeError: Table instance has no field, method, or property 'aggreate'
    Did you mean:
        Table method: 'aggregate'

In [20]:
gene_cols = [gene_col for gene_col in rows.info if gene_col.startswith('PROTEIN_CODING__')]
r1 = rows.annotate(gene_info=hl.filter(lambda x: hl.is_defined(x.genes),
                [hl.struct(genes=rows.info.get(col), predicted_consequence=col.split('__')[-1])
                 for col in gene_cols if rows.info.get(col).dtype==hl.dtype('array<str>')]))
r1.show()

+---------------+---------------+-----------------------------+----------+
| locus         | alleles       | rsid                        |     qual |
+---------------+---------------+-----------------------------+----------+
| locus<GRCh38> | array<str>    | str                         |  float64 |
+---------------+---------------+-----------------------------+----------+
| chr1:10000    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_1" | 9.99e+02 |
| chr1:10000    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_2" | 9.99e+02 |
| chr1:10602    | ["N","<BND>"] | "CMG.phase1_CMG_BND_chr1_1" | 4.61e+02 |
| chr1:41950    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_3" | 9.99e+02 |
| chr1:44000    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_4" | 9.99e+02 |
| chr1:44250    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_5" | 9.99e+02 |
| chr1:51400    | ["N","<DEL>"] | "CMG.phase1_CMG_DEL_chr1_1" | 9.99e+02 |
| chr1:52600    | ["N","<CNV>"] | "CMG.phase1_CMG_CNV_chr1_1" | 9.99e+02 |
| chr1:66234    | ["N","<BND>"] | "CMG.phase1_CMG_BND_chr1_2" | 8.07e+02 |
| chr1:66350    | ["N","<DEL>"] | "CMG.phase1_CMG_DEL_chr1_2" | 7.74e+02 |
+---------------+---------------+-----------------------------+----------+

+------------------------------------------+-----------------+-----------+--------------------+
| filters                                  | info.ALGORITHMS | info.CHR2 | info.CPX_INTERVALS |
+------------------------------------------+-----------------+-----------+--------------------+
| set<str>                                 | array<str>      | str       | array<str>         |
+------------------------------------------+-----------------+-----------+--------------------+
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"UNRESOLVED","UNSTABLE_AF_PCRMINUS"}    | ["manta"]       | "chr12"   | NA                 |
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"UNSTABLE_AF_PCRMINUS","LOW_CALL_RATE"} | ["depth"]       | "chr1"    | NA                 |
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"FAIL_minGQ"}                           | ["depth"]       | "chr1"    | NA                 |
| {"UNRESOLVED"}                           | ["manta"]       | "chr19"   | NA                 |
| {"FAIL_OUTLIER_REMOVAL"}                 | ["manta"]       | "chr1"    | NA                 |
+------------------------------------------+-----------------+-----------+--------------------+

+---------------+----------+-----------+---------------+-------------+--------------+------------+
| info.CPX_TYPE | info.END | info.END2 | info.EVIDENCE | info.SOURCE | info.STRANDS | info.SVLEN |
+---------------+----------+-----------+---------------+-------------+--------------+------------+
| str           |    int32 |     int32 | array<str>    | str         | str          |      int32 |
+---------------+----------+-----------+---------------+-------------+--------------+------------+
| NA            |    17000 |        NA | ["RD"]        | NA          | NA           |       7000 |
| NA            |    53500 |        NA | ["BAF","RD"]  | NA          | NA           |      43500 |
| NA            |    10602 |     10546 | ["SR"]        | NA          | "+-"         |         -1 |
| NA            |    52000 |        NA | ["BAF","RD"]  | NA          | NA           |      10050 |
| NA            |    66000 |        NA | ["RD"]        | NA          | NA           |      22000 |
| NA            |   116000 |        NA | ["BAF","RD"]  | NA          | NA           |      71750 |
| NA            |    64000 |        NA | ["RD"]        | NA          | NA           |      12600 |
| NA            |    58000 |        NA | ["RD"]        | NA          | NA 

In [21]:
rows.filter(rows.rsid=='CMG.phase1_CMG_INS_chr1_3').show()

+---------------+----------------------+-----------------------------+----------+----------------+
| locus         | alleles              | rsid                        |     qual | filters        |
+---------------+----------------------+-----------------------------+----------+----------------+
| locus<GRCh38> | array<str>           | str                         |  float64 | set<str>       |
+---------------+----------------------+-----------------------------+----------+----------------+
| chr1:883440   | ["N","<INS:ME:ALU>"] | "CMG.phase1_CMG_INS_chr1_3" | 3.25e+02 | {"FAIL_minGQ"} |
+---------------+----------------------+-----------------------------+----------+----------------+

+-----------------+-----------+--------------------+---------------+----------+-----------+
| info.ALGORITHMS | info.CHR2 | info.CPX_INTERVALS | info.CPX_TYPE | info.END | info.END2 |
+-----------------+-----------+--------------------+---------------+----------+-----------+
| array<str>      | str       | array<str>         | str           |    int32 |     int32 |
+-----------------+-----------+--------------------+---------------+----------+-----------+
| ["melt"]        | "chr1"    | NA                 | NA            |   883491 |        NA |
+-----------------+-----------+--------------------+---------------+----------+-----------+

+---------------+-------------+--------------+------------+-------------+----------------------+
| info.EVIDENCE | info.SOURCE | info.STRANDS | info.SVLEN | info.SVTYPE | info.UNRESOLVED_TYPE |
+---------------+-------------+--------------+------------+-------------+----------------------+
| array<str>    | str         | str          |      int32 | str         | str                  |
+---------------+-------------+--------------+------------+-------------+----------------------+
| ["SR"]        | NA          | NA           |        281 | "INS"       | NA                   |
+---------------+-------------+--------------+------------+-------------+----------------------+

+--------------------------+-------------------+------------------------------+
| info.PROTEIN_CODING__LOF | info.LINCRNA__LOF | info.PROTEIN_CODING__DUP_LOF |
+--------------------------+-------------------+------------------------------+
| array<str>               | array<str>        | array<str>                   |
+--------------------------+-------------------+------------------------------+
| NA                       | NA                | NA                           |
+--------------------------+-------------------+------------------------------+

+-----------------------+--------------------------------+-------------------------+
| info.LINCRNA__DUP_LOF | info.PROTEIN_CODING__COPY_GAIN | info.LINCRNA__COPY_GAIN |
+-----------------------+--------------------------------+-------------------------+
| array<str>            | array<str>                     | array<str>              |
+-----------------------+--------------------------------+-------------------------+
| NA                    | NA                             | NA                      |
+-----------------------+--------------------------------+-------------------------+

+----------------------------------+---------------------------+-----------------------------------+
| info.PROTEIN_CODING__DUP_PARTIAL | info.LINCRNA__DUP_PARTIAL | info.PROTEIN_CODING__MSV_EXON_OVR |
+----------------------------------+---------------------------+-----------------------------------+
| array<str>                       | array<str>                | array<str>                        |
+----------------------------------+---------------------------+-----------------------------------+
| NA                               | NA                        | NA                                |
+----------------------------------+---------------------------+-----------------------------------+

+----------------------------+-------------------------------+------------------------+
| info.LINCRNA__MSV_EXON_OVR

In [23]:
rows.filter(rows.filters.contains('FAIL_minGQ')).show()

+---------------+---------------+------------------------------+----------+----------------+
| locus         | alleles       | rsid                         |     qual | filters        |
+---------------+---------------+------------------------------+----------+----------------+
| locus<GRCh38> | array<str>    | str                          |  float64 | set<str>       |
+---------------+---------------+------------------------------+----------+----------------+
| chr1:52600    | ["N","<CNV>"] | "CMG.phase1_CMG_CNV_chr1_1"  | 9.99e+02 | {"FAIL_minGQ"} |
| chr1:67000    | ["N","<CNV>"] | "CMG.phase1_CMG_CNV_chr1_2"  | 9.99e+02 | {"FAIL_minGQ"} |
| chr1:132500   | ["N","<CNV>"] | "CMG.phase1_CMG_CNV_chr1_3"  | 9.99e+02 | {"FAIL_minGQ"} |
| chr1:181000   | ["N","<CNV>"] | "CMG.phase1_CMG_CNV_chr1_4"  | 9.99e+02 | {"FAIL_minGQ"} |
| chr1:181387   | ["N","<DEL>"] | "CMG.phase1_CMG_DEL_chr1_6"  | 6.37e+02 | {"FAIL_minGQ"} |
| chr1:181725   | ["N","<DEL>"] | "CMG.phase1_CMG_DEL_chr1_7"  | 6.24e+02 | {"FAIL_minGQ"} |
| chr1:196500   | ["N","<CNV>"] | "CMG.phase1_CMG_CNV_chr1_5"  | 9.99e+02 | {"FAIL_minGQ"} |
| chr1:284000   | ["N","<CNV>"] | "CMG.phase1_CMG_CNV_chr1_6"  | 9.99e+02 | {"FAIL_minGQ"} |
| chr1:823278   | ["N","<DEL>"] | "CMG.phase1_CMG_DEL_chr1_13" | 6.74e+02 | {"FAIL_minGQ"} |
| chr1:863579   | ["N","<BND>"] | "CMG.phase1_CMG_BND_chr1_12" | 4.61e+02 | {"FAIL_minGQ"} |
+---------------+---------------+------------------------------+----------+----------------+

+-----------------+-----------+--------------------+---------------+----------+-----------+
| info.ALGORITHMS | info.CHR2 | info.CPX_INTERVALS | info.CPX_TYPE | info.END | info.END2 |
+-----------------+-----------+--------------------+---------------+----------+-----------+
| array<str>      | str       | array<str>         | str           |    int32 |     int32 |
+-----------------+-----------+--------------------+---------------+----------+-----------+
| ["depth"]       | "chr1"    | NA                 | NA            |    58000 |        NA |
| ["depth"]       | "chr1"    | NA                 | NA            |    94000 |        NA |
| ["depth"]       | "chr1"    | NA                 | NA            |   140000 |        NA |
| ["depth"]       | "chr1"    | NA                 | NA            |   190000 |        NA |
| ["manta"]       | "chr1"    | NA                 | NA            |   181451 |        NA |
| ["manta"]       | "chr1"    | NA                 | NA            |   181824 |        NA |
| ["depth"]       | "chr1"    | NA                 | NA            |   202500 |        NA |
| ["depth"]       | "chr1"    | NA                 | NA            |   289666 |        NA |
| ["wham"]        | "chr1"    | NA                 | NA            |   823491 |        NA |
| ["manta"]       | "chr8"    | NA                 | NA            |   863579 |    293805 |
+-----------------+-----------+--------------------+---------------+----------+-----------+

+---------------+-------------+--------------+------------+-------------+----------------------+
| info.EVIDENCE | info.SOURCE | info.STRANDS | info.SVLEN | info.SVTYPE | info.UNRESOLVED_TYPE |
+---------------+-------------+--------------+------------+-------------+----------------------+
| array<str>    | str         | str          |      int32 | str         | str                  |
+---------------+-------------+--------------+------------+-------------+----------------------+
| ["RD"]        | NA          | NA           |       5400 | "CNV"       | NA                   |
| ["RD"]        | NA          | NA           |      27000 | "CNV"       | NA                   |
| ["BAF","RD"]  | NA          | NA           |       7500 | "CNV"       | NA                   |
| ["BAF","RD"]  | NA          | NA           |       9000 | "CNV"       | NA                   |
| ["SR"]        | NA          | NA           |         64 | "DEL"       | NA                   |
| ["SR"]        | NA          | NA           |         99 | "DEL"   

In [24]:
rows.filter(rows.filters.contains('PASS')).show()

+---------------+------------+------+---------+----------+-----------------+-----------+
| locus         | alleles    | rsid |    qual | filters  | info.ALGORITHMS | info.CHR2 |
+---------------+------------+------+---------+----------+-----------------+-----------+
| locus<GRCh38> | array<str> | str  | float64 | set<str> | array<str>      | str       |
+---------------+------------+------+---------+----------+-----------------+-----------+
+---------------+------------+------+---------+----------+-----------------+-----------+

+--------------------+---------------+----------+-----------+---------------+-------------+
| info.CPX_INTERVALS | info.CPX_TYPE | info.END | info.END2 | info.EVIDENCE | info.SOURCE |
+--------------------+---------------+----------+-----------+---------------+-------------+
| array<str>         | str           |    int32 |     int32 | array<str>    | str         |
+--------------------+---------------+----------+-----------+---------------+-------------+
+--------------------+---------------+----------+-----------+---------------+-------------+

+--------------+------------+-------------+----------------------+--------------------------+
| info.STRANDS | info.SVLEN | info.SVTYPE | info.UNRESOLVED_TYPE | info.PROTEIN_CODING__LOF |
+--------------+------------+-------------+----------------------+--------------------------+
| str          |      int32 | str         | str                  | array<str>               |
+--------------+------------+-------------+----------------------+--------------------------+
+--------------+------------+-------------+----------------------+--------------------------+

+-------------------+------------------------------+-----------------------+
| info.LINCRNA__LOF | info.PROTEIN_CODING__DUP_LOF | info.LINCRNA__DUP_LOF |
+-------------------+------------------------------+-----------------------+
| array<str>        | array<str>                   | array<str>            |
+-------------------+------------------------------+-----------------------+
+-------------------+------------------------------+-----------------------+

+--------------------------------+-------------------------+----------------------------------+
| info.PROTEIN_CODING__COPY_GAIN | info.LINCRNA__COPY_GAIN | info.PROTEIN_CODING__DUP_PARTIAL |
+--------------------------------+-------------------------+----------------------------------+
| array<str>                     | array<str>              | array<str>                       |
+--------------------------------+-------------------------+----------------------------------+
+--------------------------------+-------------------------+----------------------------------+

+---------------------------+-----------------------------------+----------------------------+
| info.LINCRNA__DUP_PARTIAL | info.PROTEIN_CODING__MSV_EXON_OVR | info.LINCRNA__MSV_EXON_OVR |
+---------------------------+-----------------------------------+----------------------------+
| array<str>                | array<str>                        | array<str>                 |
+---------------------------+-----------------------------------+----------------------------+
+---------------------------+-----------------------------------+----------------------------+

+-------------------------------+------------------------+-------------------------------+
| info.PROTEIN_CODING__INTRONIC | info.LINCRNA__INTRONIC | info.PROTEIN_CODING__INV_SPAN |
+-------------------------------+------------------------+-------------------------------+
| array<str>                    | array<str>             | array<str>                    |
+-------------------------------+------------------------+-------------------------------+
+-------------------------------+------------------------+-------------------------------+

+------------------------+--------------------------+-------------------+---------------------+
| info.LINCRNA__INV_SPAN | info.PROTEIN_CODING__UTR | info.LINCRNA__UTR | info.NONCODING_SPAN

In [30]:
filters = hl.array(hl.filter(lambda x: x != 'UNRESOLVED', rows.filters))
filters.dtype

dtype('array<str>')

In [49]:
mt2.rows().show()

+---------------+---------------+-----------------------------+----------+
| locus         | alleles       | rsid                        |     qual |
+---------------+---------------+-----------------------------+----------+
| locus<GRCh38> | array<str>    | str                         |  float64 |
+---------------+---------------+-----------------------------+----------+
| chr1:10000    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_1" | 9.99e+02 |
| chr1:10000    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_2" | 9.99e+02 |
| chr1:10602    | ["N","<BND>"] | "CMG.phase1_CMG_BND_chr1_1" | 4.61e+02 |
| chr1:41950    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_3" | 9.99e+02 |
| chr1:44000    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_4" | 9.99e+02 |
| chr1:44250    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_5" | 9.99e+02 |
| chr1:51400    | ["N","<DEL>"] | "CMG.phase1_CMG_DEL_chr1_1" | 9.99e+02 |
| chr1:66234    | ["N","<BND>"] | "CMG.phase1_CMG_BND_chr1_2" | 8.07e+02 |
| chr1:66350    | ["N","<DEL>"] | "CMG.phase1_CMG_DEL_chr1_2" | 7.74e+02 |
| chr1:66531    | ["N","<INS>"] | "CMG.phase1_CMG_INS_chr1_1" | 4.00e+02 |
+---------------+---------------+-----------------------------+----------+

+------------------------------------------+-----------------+-----------+--------------------+
| filters                                  | info.ALGORITHMS | info.CHR2 | info.CPX_INTERVALS |
+------------------------------------------+-----------------+-----------+--------------------+
| set<str>                                 | array<str>      | str       | array<str>         |
+------------------------------------------+-----------------+-----------+--------------------+
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"UNRESOLVED","UNSTABLE_AF_PCRMINUS"}    | ["manta"]       | "chr12"   | NA                 |
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"UNSTABLE_AF_PCRMINUS","LOW_CALL_RATE"} | ["depth"]       | "chr1"    | NA                 |
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"UNRESOLVED"}                           | ["manta"]       | "chr19"   | NA                 |
| {"FAIL_OUTLIER_REMOVAL"}                 | ["manta"]       | "chr1"    | NA                 |
| {}                                       | ["manta"]       | "chr1"    | NA                 |
+------------------------------------------+-----------------+-----------+--------------------+

+---------------+----------+-----------+---------------+-------------+--------------+------------+
| info.CPX_TYPE | info.END | info.END2 | info.EVIDENCE | info.SOURCE | info.STRANDS | info.SVLEN |
+---------------+----------+-----------+---------------+-------------+--------------+------------+
| str           |    int32 |     int32 | array<str>    | str         | str          |      int32 |
+---------------+----------+-----------+---------------+-------------+--------------+------------+
| NA            |    17000 |        NA | ["RD"]        | NA          | NA           |       7000 |
| NA            |    53500 |        NA | ["BAF","RD"]  | NA          | NA           |      43500 |
| NA            |    10602 |     10546 | ["SR"]        | NA          | "+-"         |         -1 |
| NA            |    52000 |        NA | ["BAF","RD"]  | NA          | NA           |      10050 |
| NA            |    66000 |        NA | ["RD"]        | NA          | NA           |      22000 |
| NA            |   116000 |        NA | ["BAF","RD"]  | NA          | NA           |      71750 |
| NA            |    64000 |        NA | ["RD"]        | NA          | NA           |      12600 |
| NA            |    66234 |    108051 | ["PE"]        | NA          | "-+

In [52]:
mt2.rows().samples.filter(lambda x: x.cn>4).show()

,,
locus,alleles,
locus<GRCh38>,array<str>,"array<struct{id: str, gq: int32, num_alt: int32, cn: int32}>"
chr1:10000,"[""N"",""<DUP>""]",[]
chr1:10000,"[""N"",""<DUP>""]",[]
chr1:10602,"[""N"",""<BND>""]",[]
chr1:41950,"[""N"",""<DUP>""]",[]
chr1:44000,"[""N"",""<DUP>""]",[]
chr1:44250,"[""N"",""<DUP>""]",[]
chr1:51400,"[""N"",""<DEL>""]",[]
chr1:66234,"[""N"",""<BND>""]",[]


In [44]:
alleles = mt1.GT[0] + mt1.GT[1]
alleles.show()

,,,,,,
,,'HK010_0026','HK015_0037','HK031_0079','HK031_0080','HK069-0178_1'
locus,alleles,,,,,
locus<GRCh38>,array<str>,int32,int32,int32,int32,int32
chr1:10000,"[""N"",""<DUP>""]",1,1,1,2,1
chr1:10000,"[""N"",""<DUP>""]",0,2,0,1,0
chr1:10602,"[""N"",""<BND>""]",0,0,0,0,0
chr1:41950,"[""N"",""<DUP>""]",0,0,0,0,0
chr1:44000,"[""N"",""<DUP>""]",0,0,0,1,0
chr1:44250,"[""N"",""<DUP>""]",0,0,0,0,0
chr1:51400,"[""N"",""<DEL>""]",0,1,0,0,1


In [37]:
rows = mt1.rows()
rows = rows.annotate(_filters=hl.array(hl.filter(lambda x: x != 'UNRESOLVED', rows.filters)))

In [38]:
rows = rows.annotate(__filters=hl.if_else(hl.len(rows._filters)>0, rows._filters, hl.null(hl.dtype('array<str>'))))

In [39]:
rows.show()

+---------------+---------------+-----------------------------+----------+
| locus         | alleles       | rsid                        |     qual |
+---------------+---------------+-----------------------------+----------+
| locus<GRCh38> | array<str>    | str                         |  float64 |
+---------------+---------------+-----------------------------+----------+
| chr1:10000    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_1" | 9.99e+02 |
| chr1:10000    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_2" | 9.99e+02 |
| chr1:10602    | ["N","<BND>"] | "CMG.phase1_CMG_BND_chr1_1" | 4.61e+02 |
| chr1:41950    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_3" | 9.99e+02 |
| chr1:44000    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_4" | 9.99e+02 |
| chr1:44250    | ["N","<DUP>"] | "CMG.phase1_CMG_DUP_chr1_5" | 9.99e+02 |
| chr1:51400    | ["N","<DEL>"] | "CMG.phase1_CMG_DEL_chr1_1" | 9.99e+02 |
| chr1:52600    | ["N","<CNV>"] | "CMG.phase1_CMG_CNV_chr1_1" | 9.99e+02 |
| chr1:66234    | ["N","<BND>"] | "CMG.phase1_CMG_BND_chr1_2" | 8.07e+02 |
| chr1:66350    | ["N","<DEL>"] | "CMG.phase1_CMG_DEL_chr1_2" | 7.74e+02 |
+---------------+---------------+-----------------------------+----------+

+------------------------------------------+-----------------+-----------+--------------------+
| filters                                  | info.ALGORITHMS | info.CHR2 | info.CPX_INTERVALS |
+------------------------------------------+-----------------+-----------+--------------------+
| set<str>                                 | array<str>      | str       | array<str>         |
+------------------------------------------+-----------------+-----------+--------------------+
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"UNRESOLVED","UNSTABLE_AF_PCRMINUS"}    | ["manta"]       | "chr12"   | NA                 |
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"UNSTABLE_AF_PCRMINUS","LOW_CALL_RATE"} | ["depth"]       | "chr1"    | NA                 |
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"LOW_CALL_RATE"}                        | ["depth"]       | "chr1"    | NA                 |
| {"FAIL_minGQ"}                           | ["depth"]       | "chr1"    | NA                 |
| {"UNRESOLVED"}                           | ["manta"]       | "chr19"   | NA                 |
| {"FAIL_OUTLIER_REMOVAL"}                 | ["manta"]       | "chr1"    | NA                 |
+------------------------------------------+-----------------+-----------+--------------------+

+---------------+----------+-----------+---------------+-------------+--------------+------------+
| info.CPX_TYPE | info.END | info.END2 | info.EVIDENCE | info.SOURCE | info.STRANDS | info.SVLEN |
+---------------+----------+-----------+---------------+-------------+--------------+------------+
| str           |    int32 |     int32 | array<str>    | str         | str          |      int32 |
+---------------+----------+-----------+---------------+-------------+--------------+------------+
| NA            |    17000 |        NA | ["RD"]        | NA          | NA           |       7000 |
| NA            |    53500 |        NA | ["BAF","RD"]  | NA          | NA           |      43500 |
| NA            |    10602 |     10546 | ["SR"]        | NA          | "+-"         |         -1 |
| NA            |    52000 |        NA | ["BAF","RD"]  | NA          | NA           |      10050 |
| NA            |    66000 |        NA | ["RD"]        | NA          | NA           |      22000 |
| NA            |   116000 |        NA | ["BAF","RD"]  | NA          | NA           |      71750 |
| NA            |    64000 |        NA | ["RD"]        | NA          | NA           |      12600 |
| NA            |    58000 |        NA | ["RD"]        | NA          | NA 

In [40]:
row = rows.take(1)

In [42]:
pprint(row)

[{'__filters': ['LOW_CALL_RATE'],
  '_filters': ['LOW_CALL_RATE'],
  'alleles': ['N', '<DUP>'],
  'filters': {'LOW_CALL_RATE'},
  'info': {'AC': [370],
           'AF': [0.259104],
           'ALGORITHMS': ['depth'],
           'AN': 1428,
           'CHR2': 'chr1',
           'CPX_INTERVALS': None,
           'CPX_TYPE': None,
           'END': 17000,
           'END2': None,
           'EVIDENCE': ['RD'],
           'FEMALE_AC': [156],
           'FEMALE_AF': [0.237805],
           'FEMALE_AN': 656,
           'FEMALE_FREQ_HET': 0.310976,
           'FEMALE_FREQ_HOMALT': 0.0823171,
           'FEMALE_FREQ_HOMREF': 0.606707,
           'FEMALE_N_BI_GENOS': 328,
           'FEMALE_N_HET': 102,
           'FEMALE_N_HOMALT': 27,
           'FEMALE_N_HOMREF': 199,
           'FREQ_HET': 0.319328,
           'FREQ_HOMALT': 0.0994398,
           'FREQ_HOMREF': 0.581232,
           'LINCRNA__COPY_GAIN': None,
           'LINCRNA__DUP_LOF': None,
           'LINCRNA__DUP_PARTIAL': None,
     

In [92]:
a=hl.struct(type='a',chrom='b', start=1, end=2)
a.dtype

dtype('struct{type: str, chrom: str, start: int32, end: int32}')

In [63]:
a=hl.literal([1,2,3,4,5,6])
b=[hl.if_else(a[i]%2==0, a[i], hl.null(hl.tint)) for i in range(6)]
hl.eval(b)

[None, 2, None, 4, None, 6]

In [68]:
hl.eval(hl.filter(lambda x: x%2==0, a))

[2, 4, 6]